In [1]:
import pandas as pd
import numpy as np

In [68]:
df = pd.read_csv("./original_data/final_medicine_inventory_sales_data.csv")
df.head(10)

,SKU,Product Name,Category,Dosage Form,Strength,Package Size,Batch Number,Manufacturing Date,Expiration Date,Unit Cost,Selling Price,Stock In,Stock Out,Stock Balance,Distributor ID,Distributor Name,Country,Distributor Type,Order Date,Quantity Ordered,Total Order Cost,Order Status,Payment Status,Shipping Date,Delivery Date,Transaction Date,Season,Holiday,Promotion,Lag_1,Lag_2,Lag_3,Moving Average,Demand Forecast
0,SKU_1,Medicine_SKU_1,Antifungal,Syrup,231mg,331ml,BN8255,2021-11-23,2022-12-20,14.33,23.94,186,62,124,DIST_16,Distributor_DIST_16,Germany,Pharmacy,2023-01-16,103,2465.82,Pending,Unpaid,2023-01-19,2023-01-27,2023-01-31,Winter,0,0,62,62,62,62.0,186
1,SKU_2,Medicine_SKU_2,Antifungal,Tablet,170mg,357ml,BN9857,2021-11-10,2023-04-25,42.15,72.11,129,43,86,DIST_21,Distributor_DIST_21,Japan,Wholesale,2023-01-27,61,4398.71,Delivered,Unpaid,2023-01-30,2023-02-08,2023-01-31,Winter,1,0,43,43,43,43.0,129
2,SKU_3,Medicine_SKU_3,Antiviral,Tablet,123mg,117ml,BN5909,2022-02-04,2024-11-19,24.47,44.61,183,61,122,DIST_38,Distributor_DIST_38,France,Wholesale,2023-01-19,113,5040.93,Delivered,Partially Paid,2023-01-21,2023-01-25,2023-01-31,Winter,0,1,61,61,61,61.0,183
3,SKU_4,Medicine_SKU_4,Antiviral,Tablet,263mg,312ml,BN8825,2021-12-24,2023-01-02,13.89,21.37,213,71,142,DIST_31,Distributor_DIST_31,USA,Pharmacy,2023-01-20,77,1645.49,Pending,Partially Paid,2023-01-25,2023-01-27,2023-01-31,Winter,0,0,71,71,71,71.0,213
4,SKU_5,Medicine_SKU_5,Antiviral,Tablet,475mg,204ml,BN4163,2022-06-24,2025-03-17,10.16,19.68,78,26,52,DIST_13,Distributor_DIST_13,France,Wholesale,2023-01-27,180,3542.40,Canceled,Unpaid,2023-01-29,2023-02-04,2023-01-31,Winter,0,1,26,26,26,26.0,78
5,SKU_6,Medicine_SKU_6,Antifungal,Tablet,323mg,420ml,BN6728,2021-11-24,2023-05-18,46.48,63.24,99,33,66,DIST_43,Distributor_DIST_43,UK,Hospital,2023-01-13,119,7525.56,Pending,Unpaid,2023-01-17,2023-01-26,2023-01-31,Winter,0,0,33,33,33,33.0,99
6,SKU_7,Medicine_SKU_7,Antifungal,Injection,391mg,392ml,BN2598,2021-11-15,2024-09-05,16.35,20.44,198,66,132,DIST_30,Distributor_DIST_30,India,Retail,2023-01-21,185,3781.40,Delivered,Paid,2023-01-24,2023-01-30,2023-01-31,Winter,0,0,66,66,66,66.0,198
7,SKU_8,Medicine_SKU_8,Antibiotic,Injection,195mg,425ml,BN9825,2021-12-18,2023-09-25,49.70,97.05,216,72,144,DIST_16,Distributor_DIST_16,India,Pharmacy,2023-01-18,147,14266.35,Pending,Paid,2023-01-20,2023-01-28,2023-01-31,Winter,0,1,72,72,72,72.0,216
8,SKU_9,Medicine_SKU_9,Antiviral,Tablet,195mg,447ml,BN4303,2021-09-19,2023-12-31,40.49,69.38,27,9,18,DIST_2,Distributor_DIST_2,France,Pharmacy,2023-01-29,22,1526.36,Delivered,Partially Paid,2023-02-02,2023-02-04,2023-01-31,Winter,0,1,9,9,9,9.0,27
9,SKU_10,Medicine_SKU_10,Antiviral,Injection,374mg,157ml,BN3836,2021-09-29,2024-03-21,47.39,90.84,96,32,64,DIST_20,Distributor_DIST_20,USA,Pharmacy,2023-01-12,55,4996.20,Shipped,Partially Paid,2023-01-16,2023-01-26,2023-01-31,Winter,0,0,32,32,32,32.0,96


In [54]:
df['Order Date'].value_counts

<bound method IndexOpsMixin.value_counts of 0       2023-01-16
1       2023-01-27
2       2023-01-19
3       2023-01-20
4       2023-01-27
           ...    
3595    2024-12-27
3596    2024-12-24
3597    2024-12-22
3598    2024-12-21
3599    2024-12-26
Name: Order Date, Length: 3600, dtype: object>

In [39]:
df.columns

Index(['SKU', 'Product Name', 'Category', 'Dosage Form', 'Strength',
       'Package Size', 'Batch Number', 'Manufacturing Date', 'Expiration Date',
       'Unit Cost', 'Selling Price', 'Stock In', 'Stock Out', 'Stock Balance',
       'Distributor ID', 'Distributor Name', 'Country', 'Distributor Type',
       'Order Date', 'Quantity Ordered', 'Total Order Cost', 'Order Status',
       'Payment Status', 'Shipping Date', 'Delivery Date', 'Transaction Date',
       'Season', 'Holiday', 'Promotion', 'Lag_1', 'Lag_2', 'Lag_3',
       'Moving Average', 'Demand Forecast'],
      dtype='object')

In [41]:
#converting date columns to datetime 

df['Manufacturing Date'] = pd.to_datetime(df['Manufacturing Date'])
df['Expiration Date'] = pd.to_datetime(df['Expiration Date'])
df['Order Date'] = pd.to_datetime(df['Order Date'])
df['Delivery Date'] = pd.to_datetime(df['Delivery Date'])
df['Shipping Date'] = pd.to_datetime(df['Shipping Date'])

In [42]:
#encoding sku column 

from sklearn.preprocessing import LabelEncoder

Le = LabelEncoder()
df['SKU'] = Le.fit_transform(df['SKU'])

In [43]:
# converting season column to numerical features 
df['Season'] = df['Season'].map ({'Spring': 0, 'Summer': 1, 'Autumn': 2, 'Winter': 3})

In [44]:
#Feature Engineering 
df['Days_Until_Expiry'] = (df['Expiration Date'] - pd.to_datetime('today')).dt.days 
df['Lead_Time'] = (df['Delivery Date'] - df['Order Date']).dt.days

In [45]:
# Select relevant columns
final_columns = [
        'SKU','Stock In', 'Stock Out','Stock Balance', 'Days_Until_Expiry', 'Lead_Time',
        'Season', 'Holiday', 'Promotion', 'Lag_1', 'Lag_2', 'Lag_3', 
        'Moving Average', 'Demand Forecast'
    ]

In [46]:
df = df[final_columns]
print (df)

      SKU  Stock In  Stock Out  Stock Balance  Days_Until_Expiry  Lead_Time  \
0       0       186         62            124               -827         11   
1      62       129         43             86               -701         12   
2      73       183         61            122               -127          6   
3      84       213         71            142               -814          7   
4      95        78         26             52                 -9          8   
...   ...       ...        ...            ...                ...        ...   
3595   52         0         87             35                328          9   
3596   53         0        100           -207                731          9   
3597   54         0         98             76                183         10   
3598   55         0         82           -194                174          8   
3599   57         0         59             76                591         11   

      Season  Holiday  Promotion  Lag_1  Lag_2  Lag

In [49]:
df['Promotion'].dtype

dtype('int64')

In [64]:
import pandas as pd

# Load CSV
df2 = pd.read_csv('predictions/multi_month_demand_forecast.csv')

# Show all columns
pd.set_option('display.max_rows', 300)

# Display DataFrame
df2

,SKU,Order Month,Order Year,Predicted Demand,Stock Balance,Reorder Needed,Reorder Quantity,Recommended Order Date
0,0,4,2025,4229.78,60,False,0.00,No Order Needed
1,1,4,2025,3795.15,-68,True,3863.15,2025-04-01
2,2,4,2025,3785.02,-25,True,3810.02,2025-04-01
3,3,4,2025,3185.60,-84,True,3269.60,2025-04-01
4,4,4,2025,3878.92,54,False,0.00,No Order Needed
5,5,4,2025,3850.80,121,False,0.00,No Order Needed
6,6,4,2025,3579.83,-147,True,3726.83,2025-04-01
7,7,4,2025,3046.07,-27,True,3073.07,2025-04-01
8,8,4,2025,3808.02,59,False,0.00,No Order Needed
9,9,4,2025,3380.00,-39,True,3419.00,2025-04-01


In [71]:
import pandas as pd

# Load the cleaned dataset
df = pd.read_csv("processed_data/medicine_inventory_cleaned.csv")

# Check for negative stock balances
negative_stock = df[df['Stock Balance'] < 0]

# Display the problematic rows
print(negative_stock)








Empty DataFrame
Columns: [SKU, Stock In, Stock Out, Stock Balance, Days_Until_Expiry, Lead_Time, Season, Holiday, Promotion, Lag_1, Lag_2, Lag_3, Moving Average, Demand Forecast, Order_Day, Order_Month, Order_Weekday, Days_Since_Last_Order]
Index: []
